# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [9]:
df = pd.read_csv("data/symptom_precaution.csv")
df.head(2)

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out


In [10]:
df1=pd.read_csv("data/dataset.csv")
df1.head(2)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df1.nunique()

Disease       41
Symptom_1     34
Symptom_2     48
Symptom_3     54
Symptom_4     50
Symptom_5     38
Symptom_6     32
Symptom_7     26
Symptom_8     21
Symptom_9     22
Symptom_10    21
Symptom_11    18
Symptom_12    11
Symptom_13     8
Symptom_14     4
Symptom_15     3
Symptom_16     3
Symptom_17     1
dtype: int64

In [12]:
symptom_columns = df1.filter(like='Symptom_').columns
df1[symptom_columns] = df1[symptom_columns].apply(lambda x: x.str.strip())

In [13]:
all_symptoms = df1.filter(like='Symptom_').stack().unique()

binary_df = pd.DataFrame(0, index=df1.index, columns=all_symptoms)


for col in df1.filter(like='Symptom_').columns:
    for symptom in all_symptoms:
        binary_df[symptom] = np.where(df1[col] == symptom, 1, binary_df[symptom])


binary_df['Disease'] = df1['Disease']

/var/folders/7n/ddychp7x7pd5qg_vtmj39t1m0000gn/T/ipykernel_11204/2673980709.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  binary_df['Disease'] = df1['Disease']


In [14]:
binary_df.head()

,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,continuous_sneezing,shivering,chills,watering_from_eyes,stomach_pain,acidity,...,foul_smell_of urine,continuous_feel_of_urine,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,Disease
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [15]:
binary_df.shape

(4920, 132)

In [36]:
weights=pd.read_csv('data/Symptom-severity.csv')
symptom_columns = weights.filter(like='Symptom_').columns
weights[symptom_columns] = weights[symptom_columns].apply(lambda x: x.str.strip())
weights_df=weights.copy()
weights_df["weight"] = weights["weight"] / weights["weight"].max()

weights_df.to_csv('data/weights_df.csv', index=False)
weights_df.head()

,Symptom,weight
0,itching,0.142857
1,skin_rash,0.428571
2,nodal_skin_eruptions,0.571429
3,continuous_sneezing,0.571429
4,shivering,0.714286


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models, regularizers


In [25]:
weight_dict = dict(zip(weights_df.iloc[:,0], weights_df.iloc[:,1]))

common_cols = [col for col in binary_df.columns if col in weight_dict]

df_weighted = binary_df.copy()
for col in common_cols:
    df_weighted[col] = binary_df[col] * weight_dict[col]

In [35]:
df_weighted.to_csv('data/df_weighted.csv', index=False)
df_weighted.head(16)

,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,continuous_sneezing,shivering,chills,watering_from_eyes,stomach_pain,acidity,...,foul_smell_of urine,continuous_feel_of_urine,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,Disease
0,0.142857,0.428571,0.571429,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
1,0.000000,0.428571,0.571429,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
2,0.142857,0.000000,0.571429,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
3,0.142857,0.428571,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
4,0.142857,0.428571,0.571429,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
5,0.000000,0.428571,0.571429,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
6,0.142857,0.000000,0.571429,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
7,0.142857,0.428571,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
8,0.142857,0.428571,0.571429,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
9,0.142857,0.428571,0.571429,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection


In [27]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(df_weighted["Disease"])
X = df_weighted.drop("Disease", axis=1)
y = y_encoded
X.shape

(4920, 131)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [29]:
num_classes = len(encoder.classes_)

model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),

    layers.Dense(128, activation="relu",
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),

    layers.Dense(64, activation="relu",
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),

    layers.Dense(num_classes, activation="softmax")
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3488 - loss: 3.3303 - val_accuracy: 0.8274 - val_loss: 2.2186
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - accuracy: 0.8599 - loss: 1.4643 - val_accuracy: 0.9987 - val_loss: 0.6078
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - accuracy: 0.9622 - loss: 0.6683 - val_accuracy: 0.9962 - val_loss: 0.4175
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.9759 - loss: 0.5257 - val_accuracy: 1.0000 - val_loss: 0.3715
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.9819 - loss: 0.4693 - val_accuracy: 1.0000 - val_loss: 0.3508
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - accuracy: 0.9902 - loss: 0.4286 - val_accuracy: 1.0000 - val_loss: 0.3318
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - accuracy: 0.9908 - loss: 0.4100 - val_accuracy: 1.0000 - val_loss: 0.3181
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.9879 - loss: 0.3913 - val_accura

In [30]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 1.0000 - loss: 0.0817
Test Accuracy: 1.0


In [99]:
model.save("my_model.keras")

In [100]:
from tensorflow.keras.models import load_model
loaded_model = load_model("my_model.keras")

/Users/onuryeniyurt/PyCharmMiscProject/.venv/lib/python3.13/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
